## Load/import packages

In [2]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

from os import listdir, mkdir
from os.path import splitext
from os.path import join

%matplotlib inline

## Pre-processing
1. Grab 1 Video at a time ✔
2. Extract frames one by one ✔
3. Apply face alignment on the image (detect face and segment and align it with e.g. **Haar cascade classifier by Viola and Jones**
4. Apply illumination normalization (histogram equalization)
5. Pose normalization
6. Store the image in a named folder same name as video. The image should be stored under the as videoname_frameXXXX ✔
7. Data augmentation to upsample some classes

## Directories
### Aff-Wild2

In [3]:
from load_filenames import (AW2_dir_train_labels,
                            AW2_dir_val_labels,
                            AW2_dir_allvideos,
                            AW2_train_FN,
                            AW2_train_FN_split,
                            AW2_val_FN,
                            AW2_val_FN_split,
                            AW2_videos_FN,
                            AW2_videos_FN_split,
                            AF7_dir_videos, 
                            AF7_dir_labels, 
                            AF7_labeled_videos_FN)

In [4]:
AW2_dir_allvideos

'D:\\Aff-Wild2 Dataset\\Aff-wild2\\Videos\\all_videos'

# Start Pre-processing

In [5]:
AW2_videos_FN

['1-30-1280x720.mp4',
 '10-60-1280x720.mp4',
 '100-29-1080x1920.mp4',
 '101-30-1080x1920.mp4',
 '102-30-640x360.mp4',
 '102.avi',
 '103-30-384x480.mp4',
 '103.avi',
 '104-17-720x480.mp4',
 '105-30-1280x720.mp4',
 '105.avi',
 '106-30-720x1280.mp4',
 '106.avi',
 '107-30-640x480.mp4',
 '107.avi',
 '108-15-640x480.mp4',
 '108.avi',
 '109-30-1280x720.mp4',
 '11-24-1920x1080.mp4',
 '110-30-270x480.mp4',
 '110.avi',
 '111-25-1920x1080.mp4',
 '111.avi',
 '112-30-640x360.mp4',
 '112.avi',
 '113-60-1280x720.mp4',
 '113.avi',
 '114-30-1280x720.mp4',
 '114.avi',
 '115-30-1280x720.mp4',
 '116-30-1280x720.mp4',
 '116.avi',
 '117-25-1920x1080.mp4',
 '117.avi',
 '118-30-640x480.mp4',
 '118.avi',
 '119-30-848x480.mp4',
 '119.avi',
 '12-24-1920x1080.mp4',
 '120-30-1280x720.mp4',
 '120.avi',
 '121-24-1920x1080.mp4',
 '121.avi',
 '122-60-1920x1080-1.mp4',
 '122-60-1920x1080-2.mp4',
 '122-60-1920x1080-3.mp4',
 '122-60-1920x1080-4.mp4',
 '122-60-1920x1080-5.mp4',
 '122.avi',
 '123-25-1920x1080.mp4',
 '123.a

In [6]:
AW2_videos_FN_split

[('1-30-1280x720', '.mp4'),
 ('10-60-1280x720', '.mp4'),
 ('100-29-1080x1920', '.mp4'),
 ('101-30-1080x1920', '.mp4'),
 ('102-30-640x360', '.mp4'),
 ('102', '.avi'),
 ('103-30-384x480', '.mp4'),
 ('103', '.avi'),
 ('104-17-720x480', '.mp4'),
 ('105-30-1280x720', '.mp4'),
 ('105', '.avi'),
 ('106-30-720x1280', '.mp4'),
 ('106', '.avi'),
 ('107-30-640x480', '.mp4'),
 ('107', '.avi'),
 ('108-15-640x480', '.mp4'),
 ('108', '.avi'),
 ('109-30-1280x720', '.mp4'),
 ('11-24-1920x1080', '.mp4'),
 ('110-30-270x480', '.mp4'),
 ('110', '.avi'),
 ('111-25-1920x1080', '.mp4'),
 ('111', '.avi'),
 ('112-30-640x360', '.mp4'),
 ('112', '.avi'),
 ('113-60-1280x720', '.mp4'),
 ('113', '.avi'),
 ('114-30-1280x720', '.mp4'),
 ('114', '.avi'),
 ('115-30-1280x720', '.mp4'),
 ('116-30-1280x720', '.mp4'),
 ('116', '.avi'),
 ('117-25-1920x1080', '.mp4'),
 ('117', '.avi'),
 ('118-30-640x480', '.mp4'),
 ('118', '.avi'),
 ('119-30-848x480', '.mp4'),
 ('119', '.avi'),
 ('12-24-1920x1080', '.mp4'),
 ('120-30-1280x720

### Things to fix in code below
- Add step 4 till 5 from preprocess pipeline

Things to note about OpenCV: https://stackoverflow.com/questions/42406338/why-cv2-imwrite-changes-the-color-of-pics/42406781

In [7]:
from skimage.io import imshow

import time

In [141]:
# Define directory to save the frames
frame_dir = r'D:\Aff-Wild2 Dataset\Aff-wild2\Videos\frames'

# Set timer to measure each loop
start = time.time()

# Initiate pre-processing for each video
for video, ext in AW2_videos_FN_split[:1]:
    count = 0
    frames_path = join(AW2_dir_allvideos, video+ext)
    cap = cv2.VideoCapture(frames_path)
    print(f"Processing: {video+ext}")

    # Create a directory with the name of the video to save all the frames
    try:
        mkdir(join(frame_dir, video))
    except FileExistsError:
        print("Directory already exists!")

    # Extract each frame from the video, the frame becomes numpy array when extracted
    # The array is row (height) x column (width) x color (3), where Color is BGR 
    while cap.isOpened():
        ret, frame = cap.read()
        if ret != True:
            break
        
        ## Source OpenCV4 with Python Blueprints
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cascade_clf = cv2.CascadeClassifier(r'C:\Users\ion\anaconda3\envs\tf-gpu\Lib\site-packages\cv2\data\haarcascade_frontalface_alt2.xml')
        faces = cascade_clf.detectMultiScale(gray_frame,
                                             scaleFactor=1.2, 
                                             minNeighbors=5, 
                                             flags=cv2.CASCADE_SCALE_IMAGE)
        
        for x,y,w,h in faces:
            head = frame[y:y + h, x:x + w]
#             frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            
        filename = "frame_%d.jpg" % count
        save_path = join(join(frame_dir, video), filename)
        count += 1

        # Write the frame to directory
        if not cv2.imwrite(save_path, head):
            raise Exception("Could not write image")
    cap.release()
    print(f"Done! Time ran since start: {round(time.time()-start)//60 }:{round(time.time()-start)%60}")

Processing: 1-30-1280x720.mp4
Directory already exists!
Done! Time ran since start: 21:21


*** 

In [129]:
img = cv2.imread(r'C:\Users\ion\Desktop\test.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cascade_clf = cv2.CascadeClassifier(r'C:\Users\ion\anaconda3\envs\tf-gpu\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
faces = cascade_clf.detectMultiScale(gray_img,
                                     scaleFactor=1.2, 
                                     minNeighbors=5, 
                                     flags=cv2.CASCADE_SCALE_IMAGE)

In [130]:
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

In [131]:
window_name = 'image'
cv2.imshow(window_name,img)
cv2.waitKey(0)

114

In [132]:
cv2.destroyAllWindows()